<a href="https://colab.research.google.com/github/GuyFarb/Cars-Project/blob/main/CNN5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# נתיבים לקבצים
train_tfrecord_path = "/content/drive/MyDrive/cars196_train.tfrecord"
test_tfrecord_path = "/content/drive/MyDrive/cars196_test.tfrecord"

# פרמטרים
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# פונקציה לפענוח TFRecord
def parse_tfrecord(example):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64)
    }

    example = tf.io.parse_single_example(example, feature_description)
    image = tf.image.decode_jpeg(example["image"], channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    label = example["label"]

    return image, label

# טעינת הדאטה
train_dataset = tf.data.TFRecordDataset(train_tfrecord_path).map(parse_tfrecord, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_dataset = tf.data.TFRecordDataset(test_tfrecord_path).map(parse_tfrecord, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# נתיבים לקבצים
train_tfrecord_path = "/content/drive/MyDrive/Car196_project/processed_cars196/cars196_train.tfrecord"
test_tfrecord_path = "/content/drive/MyDrive/Car196_project/processed_cars196/cars196_test.tfrecord"

# פרמטרים
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# פונקציה לפענוח TFRecord
def parse_tfrecord(example):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64)
    }

    example = tf.io.parse_single_example(example, feature_description)
    image = tf.image.decode_jpeg(example["image"], channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    label = example["label"]

    return image, label

# טעינת הדאטה
train_dataset = tf.data.TFRecordDataset(train_tfrecord_path).map(parse_tfrecord, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_dataset = tf.data.TFRecordDataset(test_tfrecord_path).map(parse_tfrecord, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [31]:
import tensorflow as tf

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(image, label):
    feature = {
        'image': _bytes_feature(image),
        'label': _int64_feature(label)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()


In [6]:
for example in tf.data.TFRecordDataset(tfrecord_path).take(1):
    parsed = tf.io.parse_single_example(example, {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    })
    image = tf.io.decode_raw(parsed['image'], tf.uint8)
    print("Raw image shape:", image.shape)
    print("Label:", parsed['label'].numpy())


Raw image shape: (23410,)
Label: 19


In [16]:
def parse_example(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    parsed = tf.io.parse_single_example(example_proto, feature_description)

    # במקום decode_raw → decode_image
    image = tf.image.decode_jpeg(parsed['image'], channels=3)
    image = tf.image.resize(image, [224, 224])  # שנה כאן אם צריך גודל אחר
    image = tf.cast(image, tf.float32) / 255.0  # נורמליזציה
    label = parsed['label']
    return image, label


In [ ]:
# חיבור לגוגל דרייב
from google.colab import drive
drive.mount('/content/drive')

# ספריות נדרשות
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# נתיב לקובץ TFRecord
tfrecord_path = "/content/drive/MyDrive/Colab Notebooks/cars196_train.tfrecord"

# קריאת קובץ TFRecord
def parse_example(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    parsed = tf.io.parse_single_example(example_proto, feature_description)

    # ⬅️ פענוח תמונה מ-JPEG
    image = tf.image.decode_jpeg(parsed['image'], channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0  # Normalize

    label = parsed['label']
    return image, label

# טעינת הנתונים מה-TFRecord
dataset = tf.data.TFRecordDataset(tfrecord_path).map(parse_example)

# המרה ל-NumPy (אם אתה רוצה לעבוד עם ImageDataGenerator)
images, labels = [], []
for img, lbl in dataset:
    images.append(img.numpy())
    labels.append(lbl.numpy())

images = np.array(images)
labels = np.array(labels)
labels_categorical = to_categorical(labels, num_classes=196)

print(f"✅ Images shape: {images.shape}, Labels shape: {labels.shape}")

# בניית המודל
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(196, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

model.summary()

# Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    horizontal_flip=True,
    validation_split=0.2
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

# אימון המודל
history = model.fit(
    datagen.flow(images, labels_categorical, subset='training', batch_size=64),
    validation_data=datagen.flow(images, labels_categorical, subset='validation', batch_size=64),
    epochs=20,
    callbacks=callbacks
)

# שמירת המודל
model.save("cnn_cars196_model_final.h5")
print("✅ המודל נשמר בהצלחה!")

# הערכת ביצועים
val_gen = datagen.flow(images, labels_categorical, subset='validation', batch_size=64, shuffle=False)
predictions = model.predict(val_gen)
y_pred = np.argmax(predictions, axis=1)
y_true = labels[val_gen.index_array]

report = classification_report(y_true, y_pred, digits=4, output_dict=True, zero_division=0)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv("classification_report_detailed.csv")
print(report_df.head(50))

# גרפים
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Images shape: (8144, 224, 224, 3), Labels shape: (8144,)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 111, 111, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 54, 54, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 26, 26, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 196)                 │         100,548 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 260,740 (1018.52 KB)

 Trainable params: 260,292 (1016.77 KB)

 Non-trainable params: 448 (1.75 KB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 996s 10s/step - accuracy: 0.0091 - loss: 5.2939 - top_k_categorical_accuracy: 0.0367 - val_accuracy: 0.0086 - val_loss: 5.3927 - val_top_k_categorical_accuracy: 0.0246 - learning_rate: 0.0010
Epoch 2/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 971s 10s/step - accuracy: 0.0184 - loss: 5.0900 - top_k_categorical_accuracy: 0.0776 - val_accuracy: 0.0074 - val_loss: 5.8161 - val_top_k_categorical_accuracy: 0.0276 - learning_rate: 0.0010
Epoch 3/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.0307 - loss: 4.9625 - top_k_categorical_accuracy: 0.1061
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
102/102 ━━━━━━━━━━━━━━━━━━━━ 967s 9s/step - accuracy: 0.0307 - loss: 4.9627 - top_k_categorical_accuracy: 0.1060 - val_accuracy: 0.0111 - val_loss: 5.6507 - val_top_k_categorical_accuracy: 0.0338 - learning_rate: 0.0010
Epoch 4/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 957s 9s/step - accuracy: 0.0401 - loss: 4.8505 - top_k_categorical_accuracy: 0

In [ ]:
# 🔸 חיבור ל-Drive
from google.colab import drive
drive.mount('/content/drive')

# 🔸 ייבוא ספריות
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 🔸 פונקציית פירוק לדוגמה מ-TFRecord
def parse_example(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.image.decode_jpeg(parsed['image'], channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    label = parsed['label']
    return image, label

# 🔸 קריאה מהקובץ של ה-Train
train_path = "/content/drive/MyDrive/Colab Notebooks/cars196_train.tfrecord"
train_dataset = tf.data.TFRecordDataset(train_path).map(parse_example)

# המרה ל-NumPy
images_train, labels_train = [], []
for img, lbl in train_dataset:
    images_train.append(img.numpy())
    labels_train.append(lbl.numpy())

images_train = np.array(images_train)
labels_train = np.array(labels_train)
labels_train_categorical = to_categorical(labels_train, num_classes=196)

print(f"✅ Train Images: {images_train.shape}, Labels: {labels_train.shape}")

# 🔸 בניית המודל
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(196, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

# 🔸 ImageDataGenerator עם validation split
datagen = ImageDataGenerator(
    rotation_range=10,
    horizontal_flip=True,
    validation_split=0.2
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

# 🔸 אימון
history = model.fit(
    datagen.flow(images_train, labels_train_categorical, subset='training', batch_size=64),
    validation_data=datagen.flow(images_train, labels_train_categorical, subset='validation', batch_size=64),
    epochs=20,
    callbacks=callbacks
)

# 🔸 שמירת המודל
model.save("cnn_cars196_model_final.h5")
print("✅ המודל נשמר בהצלחה!")

# ===============================================
# 🔸 🔹 🔸 שלב ה-TEST 🔸 🔹 🔸
# ===============================================

# טעינת סט הבדיקה
test_path = "/content/drive/MyDrive/Colab Notebooks/cars196_test.tfrecord"
test_dataset = tf.data.TFRecordDataset(test_path).map(parse_example)

# המרה ל-NumPy
images_test, labels_test = [], []
for img, lbl in test_dataset:
    images_test.append(img.numpy())
    labels_test.append(lbl.numpy())

images_test = np.array(images_test)
labels_test = np.array(labels_test)
labels_test_categorical = to_categorical(labels_test, num_classes=196)

print(f"🧪 Test Images: {images_test.shape}, Labels: {labels_test.shape}")

# חיזוי על סט הבדיקה
predictions_test = model.predict(images_test, batch_size=64)
y_pred = np.argmax(predictions_test, axis=1)
y_true = labels_test

# דוח ביצועים
report_test = classification_report(y_true, y_pred, digits=4, output_dict=True, zero_division=0)
report_df = pd.DataFrame(report_test).transpose()
report_df.to_csv("classification_report_test.csv")
print(report_df.head(50))

# ===============================================
# 🔸 גרף ביצועים
# ===============================================
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


Mounted at /content/drive
✅ Train Images: (8144, 224, 224, 3), Labels: (8144,)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 958s 9s/step - accuracy: 0.0059 - loss: 5.2938 - top_k_categorical_accuracy: 0.0339 - val_accuracy: 0.0037 - val_loss: 5.3952 - val_top_k_categorical_accuracy: 0.0313 - learning_rate: 0.0010
Epoch 2/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 930s 9s/step - accuracy: 0.0183 - loss: 5.0981 - top_k_categorical_accuracy: 0.0802 - val_accuracy: 0.0043 - val_loss: 5.6917 - val_top_k_categorical_accuracy: 0.0240 - learning_rate: 0.0010
Epoch 3/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.0241 - loss: 4.9886 - top_k_categorical_accuracy: 0.0979
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
102/102 ━━━━━━━━━━━━━━━━━━━━ 930s 9s/step - accuracy: 0.0241 - loss: 4.9887 - top_k_categorical_accuracy: 0.0980 - val_accuracy: 0.0043 - val_loss: 6.0175 - val_top_k_categorical_accuracy: 0.0184 - learning_rate: 0.0010
Epoch 4/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 927s 9s/step - accuracy: 0.0394 - loss: 4.8602 - top_k_categorical_accuracy: 0.1